In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
dfinstituciones = pd.read_csv('data/2023-04-11T04-36-02-388Zinstituciones_educativas.csv')
dfsunedu = pd.read_json('data/2023-04-11T04-36-09-824ZUniversidades.json')

In [3]:
dfhomologadas_pre = dfinstituciones.copy()

In [4]:
dfsunedu.head(2)

,código INEI,Nombre,Siglas
0,97,Asociación Civil Universidad de Ciencias y Hum...,UCH
1,505,Escuela de Posgrado Newman S.A.C.,EP Newman S.A.C


In [5]:
dfsinonimos = dfsunedu.loc[:,['Nombre ', 'Siglas ']]

In [6]:
dfsinonimos["list_sinonimos"] = ""

In [7]:
dfsinonimos.head(2)

,Nombre,Siglas,list_sinonimos
0,Asociación Civil Universidad de Ciencias y Hum...,UCH,
1,Escuela de Posgrado Newman S.A.C.,EP Newman S.A.C,


In [8]:
dfhomologadas_pre.head(2)

,candidateId,value
0,630633fa1e599d0009d6dee0,Ucv
1,630633fa1e599d0009d6dee0,José Gálvez


In [9]:
dfhomologadas_pre['value_upper'] = dfhomologadas_pre.loc[:,'value'].str.upper()

In [10]:
dfhomologadas_pre.head(2)

,candidateId,value,value_upper
0,630633fa1e599d0009d6dee0,Ucv,UCV
1,630633fa1e599d0009d6dee0,José Gálvez,JOSÉ GÁLVEZ


In [11]:
def formar_siglas(nombre_inst):
    siglas = ''
    list_nombres_parciales = nombre_inst.split(' ')
    if len(list_nombres_parciales)> 1:
        for nombre in list_nombres_parciales:
            if len(nombre) > 2:
                siglas = siglas+nombre[0]
    else:
        siglas = nombre_inst
    return siglas

In [12]:
dfhomologadas_pre['sigla_formada'] = dfhomologadas_pre['value_upper'].apply(lambda x:formar_siglas(x))

In [13]:
dfhomologadas_pre.head()

,candidateId,value,value_upper,sigla_formada
0,630633fa1e599d0009d6dee0,Ucv,UCV,UCV
1,630633fa1e599d0009d6dee0,José Gálvez,JOSÉ GÁLVEZ,JG
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,UNIVERSIDAD PERUANA DE CIENCIAS APLICADAS - UPC,UPCAU
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,CENTRO PERUANO DE ESTUDIOS BANCARIOS - CEPEBAN,CPEBC
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,INSTITUTO DE FORMACIÓN BANCARIA - IFB,IFBI


In [14]:
def primer_filtro_homologacion(universidad_value):
    homologacion = 0
    list_palabras = universidad_value.split(' ')
    for list_palabra in list_palabras:
        if list_palabra in list_unique_homologados:
            homologacion = 1
    return homologacion

In [15]:
list_unique_homologados = list(dfsinonimos.loc[:,"Siglas "].unique())
def segundo_filtro_homologacion(universidad_sigla):
    homologacion = 0
    if universidad_sigla in list_unique_homologados:
        homologacion = 1

    return homologacion

In [16]:
dfhomologadas_pre['universidad_homologada_1'] = dfhomologadas_pre['value_upper'].apply(lambda x:primer_filtro_homologacion(x))

In [17]:
dfhomologadas_pre.head(2)

,candidateId,value,value_upper,sigla_formada,universidad_homologada_1
0,630633fa1e599d0009d6dee0,Ucv,UCV,UCV,1
1,630633fa1e599d0009d6dee0,José Gálvez,JOSÉ GÁLVEZ,JG,0


In [18]:
dfhomologadas_pre['universidad_homologada_2'] = dfhomologadas_pre['value_upper'].apply(lambda x:segundo_filtro_homologacion(x))

In [19]:
dfhomologadas_pre.head(2)

,candidateId,value,value_upper,sigla_formada,universidad_homologada_1,universidad_homologada_2
0,630633fa1e599d0009d6dee0,Ucv,UCV,UCV,1,1
1,630633fa1e599d0009d6dee0,José Gálvez,JOSÉ GÁLVEZ,JG,0,0


In [20]:
dfhomologadas_pre['universidad homologada numero'] = dfhomologadas_pre['universidad_homologada_1'] + dfhomologadas_pre['universidad_homologada_2']

In [21]:
dfhomologadas_pre['universidad homologada'] = np.where(dfhomologadas_pre['universidad homologada numero']>0, "Si", "No")

In [22]:
dfhomologadas_pre.head()

,candidateId,value,value_upper,sigla_formada,universidad_homologada_1,universidad_homologada_2,universidad homologada numero,universidad homologada
0,630633fa1e599d0009d6dee0,Ucv,UCV,UCV,1,1,2,Si
1,630633fa1e599d0009d6dee0,José Gálvez,JOSÉ GÁLVEZ,JG,0,0,0,No
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,UNIVERSIDAD PERUANA DE CIENCIAS APLICADAS - UPC,UPCAU,1,0,1,Si
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,CENTRO PERUANO DE ESTUDIOS BANCARIOS - CEPEBAN,CPEBC,0,0,0,No
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,INSTITUTO DE FORMACIÓN BANCARIA - IFB,IFBI,0,0,0,No


In [23]:
universidades_homologadas = dfhomologadas_pre.loc[:,['candidateId','value','universidad homologada']]
universidades_homologadas.to_csv('data/universidades_homologadas.csv', index = False)

In [25]:
lista_nombres_valores = list(dfhomologadas_pre.value_upper.unique())

In [26]:
my_new_dict1 = {}

for row1 in dfsinonimos.values:
    list_temp = []
    for row2 in lista_nombres_valores:  
        if row1[1] in row2.split(' '):
            list_temp.append(row2)
            my_new_dict1[row1[1]] = list_temp

In [28]:
my_new_dict2  = {}
lista_siglas_formadas = list(dfhomologadas_pre.sigla_formada.unique())
for key in my_new_dict1:
    list_temp2 = []
    for rows in dfhomologadas_pre.values:
        if key in rows[3]:
            list_temp2.append(rows[1])
            my_new_dict2[key] = list_temp2

In [29]:
len(my_new_dict2.get('UCV'))

386

In [30]:
len(my_new_dict1.get('UCV'))

4

In [32]:
for key, value in my_new_dict2.items():
    for df in my_new_dict1.get(key):
        value.append(df)

In [33]:
for key, value in my_new_dict2.items():
    my_new_dict2[key] = list(set(my_new_dict2[key]))

In [35]:
dfsinonimos['list_sinonimos'] = dfsinonimos['Siglas '].map(my_new_dict2)

In [36]:
dfsinonimos.head() 

,Nombre,Siglas,list_sinonimos
0,Asociación Civil Universidad de Ciencias y Hum...,UCH,"[Uch Ciencias Y Humanidades, Universidad De Ci..."
1,Escuela de Posgrado Newman S.A.C.,EP Newman S.A.C,NaN
2,Escuela de Postgrado Gerens S.A.,GERENS,NaN
3,Facultad de Teología Pontificia y Civil de Lima,FTPCL,NaN
4,Pontificia Universidad Católica del Perú,PUCP,"[PUCP, IDIOMAS CATÓLICA PUCP, Pontificia Unive..."


In [37]:
dfsinonimos_final = dfsinonimos.loc[dfsinonimos['list_sinonimos'].notnull()]

In [41]:
dfsinonimos_final.loc[:,['Nombre ', 'list_sinonimos']].to_json('data/universidad_sinonimos.json')